# Project setup and imports
In this section I will setup the environment and define project paths.Using pathlib makes my notebook portable so that it works on any computer or github repo.

In [1]:
#Imports

from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
# Define base paths
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
Raw_Data  = PROJECT_ROOT / "data" / "raw_data"
Clean_Data = PROJECT_ROOT / "data" / "clean_data"

# 1) Load the  `fact_orders` dataset and Initial Data Understanding
Here, I will load the main orders dataset which contains one row per customer order.  
This is the core fact table of the analysis.
Before cleaning, will I inspect the dataset to understand its structure:
- How many rows and columns?
- What data types exist?
- Are there missing values?
- What does a typical record look like?

In [86]:
# reading the dataset

orders = pd.read_csv(Raw_Data/"fact_orders.csv")


In [87]:
# Display first few rows
orders.head() 

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N


In [88]:
#display number of rows and columns
print("Rows, Columns:", orders.shape)

Rows, Columns: (149166, 11)


In [89]:
#display datatypes and on null values of each columns

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149166 entries, 0 to 149165
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             149166 non-null  object 
 1   customer_id          149166 non-null  object 
 2   restaurant_id        149166 non-null  object 
 3   delivery_partner_id  143531 non-null  object 
 4   order_timestamp      149166 non-null  object 
 5   subtotal_amount      149166 non-null  float64
 6   discount_amount      149166 non-null  float64
 7   delivery_fee         149166 non-null  float64
 8   total_amount         149166 non-null  float64
 9   is_cod               149166 non-null  object 
 10  is_cancelled         149166 non-null  object 
dtypes: float64(4), object(7)
memory usage: 12.5+ MB


In [90]:
#find missing values

orders.isna().sum()


order_id                  0
customer_id               0
restaurant_id             0
delivery_partner_id    5635
order_timestamp           0
subtotal_amount           0
discount_amount           0
delivery_fee              0
total_amount              0
is_cod                    0
is_cancelled              0
dtype: int64

In [91]:
#display statistical data of each numeric columns
orders.describe()

,subtotal_amount,discount_amount,delivery_fee,total_amount
count,149166.000000,149166.000000,149166.000000,149166.000000
mean,314.373203,18.891589,30.069224,325.550833
std,147.346779,29.801279,10.997816,146.790190
min,0.000000,0.000000,0.000000,0.000000
25%,234.960000,0.000000,24.730000,249.842500
50%,309.070000,0.000000,31.485000,321.700000
75%,396.290000,34.690000,38.240000,406.590000
max,900.000000,222.530000,45.000000,944.910000


In [92]:
#checking for duplicates
duplicates_order_id = orders["order_id"].duplicated().sum()
print("Duplicate Order IDs:", duplicates_order_id)

Duplicate Order IDs: 0


## Observations
- The dataset has 149166 rows and 11 columns.  
- `order_timestamp` is stored as text (object) needs conversion to datetime datatype.  
- 5635 values are missing in `delivery_partner_id` need to check that column in depth.
- We will convert Y/N in columns `is_cod` and `is_cancelled` to boolean datatype.

# Data Cleaning
Here I will clean and prepare the data for analysis:
1) we will do missing  vaue analysis
2) datatype cleaning


# Missing value analysis

In [93]:
# Select only rows where delivery_partner_id is missing
missing_deliveries = orders[orders["delivery_partner_id"].isna()]

In [94]:
# See how many are missing
len(missing_deliveries)

5635

In [95]:
missing_deliveries.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
15,ORD202501011873,CUST050526,REST19508,NaN,2025-01-01 12:10:00,0.0,0.0,0.0,0.0,N,Y
24,ORD202501022002,CUST114265,REST08397,NaN,2025-01-01 12:15:00,0.0,0.0,0.0,0.0,N,Y
50,ORD202501012168,CUST119646,REST12616,NaN,2025-01-01 12:29:00,0.0,0.0,0.0,0.0,N,Y
54,ORD202501023485,CUST056936,REST03176,NaN,2025-01-01 12:32:00,0.0,0.0,0.0,0.0,N,Y
57,ORD202501000523,CUST195834,REST10055,NaN,2025-01-01 12:33:00,0.0,0.0,0.0,0.0,N,Y


### From intial analysis it looks all missing delivery partner id are because they were cancelled orders and hence were never assigned

In [96]:
# Lets do deep analysis. How many of these missing rows were cancelled?
missing_deliveries["is_cancelled"].value_counts()

is_cancelled
Y    5635
Name: count, dtype: int64

#### All of these missing delivery id are cancelled and hence we will replace it with "Cancelled before assignment"

In [97]:
#replacing null as "Cancelled before assignment"
orders["delivery_partner_id"] = orders["delivery_partner_id"].fillna("Cancelled before assignment")


### Observations:- Missing Value Analysis — `delivery_partner_id`
- Found 5635 missing values (~3.8% of data).
- All of these correspond to cancelled orders (no delivery partner assigned).
- Conclusion: Missing values are not random; they indicate orders cancelled before rider assignment.
- Decision: Keep these rows, and fill `delivery_partner_id` with `"Cancelled before assignment"` for clarity.

# Datatype cleaning
Now that missing values are handled, I’ll ensure each column has the correct data type.
This step is important for further time-series and categorical analysis.

In [98]:
# Make a working copy
orders_clean = orders.copy()

In [99]:
# Convert order_timestamp to datetime
orders_clean["order_timestamp"] = pd.to_datetime(orders_clean["order_timestamp"], errors="coerce")

In [100]:
#add a column order date which has pure date and no time
orders_clean["order_date"] = pd.to_datetime(orders_clean["order_timestamp"].dt.date)

In [101]:
# Convert Y/N columns to Boolean (True/False)
for col in ["is_cod", "is_cancelled"]:
    orders_clean[col] = orders_clean[col].astype(str).str.strip().str.upper().map({"Y": True, "N": False})


In [102]:
# Confirm data types
orders_clean.dtypes

order_id                       object
customer_id                    object
restaurant_id                  object
delivery_partner_id            object
order_timestamp        datetime64[ns]
subtotal_amount               float64
discount_amount               float64
delivery_fee                  float64
total_amount                  float64
is_cod                           bool
is_cancelled                     bool
order_date             datetime64[ns]
dtype: object

In [103]:
# confirm dataset head
orders_clean.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled,order_date
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,False,False,2025-01-01
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,True,False,2025-01-01
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,False,False,2025-01-01
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,False,False,2025-01-01
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,False,False,2025-01-01


### Observations
- `order_timestamp` successfully converted to datetime.
- `order_date` column is added and is used as datatype datetime.
- Both `is_cod` and `is_cancelled` standardized to True/False for easy analysis.
- No unexpected nulls introduced during conversion.

This ensures all time-based and boolean operations will behave correctly.

# Add Crisis Phase Column

QuickBite faced a crisis between **June17, 2025**.  
To compare customer behavior and order trends across time,  
we’ll classify each order into one wohree phases:
- **Pre-crisis** → before Jne 10, 2025  
- **Cris ry** → afterJune 17, 2025

In [104]:
# Define date boundaries
cut1 = pd.Timestamp("2025-06-01")

# Function to label each order
conditions = [
    orders_clean["order_timestamp"] < cut1,
    orders_clean["order_timestamp"] > cut1
]
choices = ["Pre-crisis", "Crisis"]

orders_clean["phase"] = np.select(conditions, choices, default=pd.NA)

# Quick check
orders_clean["phase"].value_counts(dropna=False)


phase
Pre-crisis    113806
Crisis         35360
Name: count, dtype: int64

### Observations
- Each order is now classified as **Pre-crisis**, **Crisis**.
- This will help analyze how metrics (orders, cancellations, spending) change across these time periods.

# Save the Clean Dataset

In [105]:
# Save cleaned file in CSV format
orders_clean.to_csv(Clean_Data / "fact_orders_clean.csv", index=False)

# 2) Understanding and Cleaning the `dim_customer` Dataset

Now we will move to the **Customer Dimension Table (`dim_customer`)**.  
This dataset contains customer-level details such as city, signup date, and acquisition channel.

These fields will help us:
- Segment customers by location and acquisition method  
- Analyze growth trends over time  
- Later we can join with order data for retention and engagement analysis

In [106]:
# Load the customer dataset
customers = pd.read_csv(Raw_Data / "dim_customer.csv")

In [107]:
# Display first few rows
customers.head() 

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,21-03-2025,Pune,Organic
1,CUST000008,07-02-2025,Kolkata,Referral
2,CUST000009,25-02-2025,Delhi,Paid
3,CUST000010,28-01-2025,Hyderabad,Paid
4,CUST000011,18-01-2025,Kolkata,Organic


In [108]:
#display number of rows and columns
print("Rows, Columns:", customers.shape)

Rows, Columns: (107776, 4)


In [109]:
#display datatypes and non null values of each columns
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107776 entries, 0 to 107775
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   customer_id          107776 non-null  object
 1   signup_date          107776 non-null  object
 2   city                 107776 non-null  object
 3   acquisition_channel  107776 non-null  object
dtypes: object(4)
memory usage: 3.3+ MB


In [110]:
#find missing values

customers.isna().sum()

customer_id            0
signup_date            0
city                   0
acquisition_channel    0
dtype: int64

In [111]:
#display statistical data of each numeric columns
customers.describe()

,customer_id,signup_date,city,acquisition_channel
count,107776,107776,107776,107776
unique,107776,333,8,4
top,CUST000007,31-12-2024,Bengaluru,Organic
freq,1,736,30281,55382


In [112]:
#checking for duplicates
duplicates_customer_id = customers["customer_id"].duplicated().sum()
print("Duplicate Customer IDs:", duplicates_customer_id)

Duplicate Customer IDs: 0


# Observations
- The dataset has 107776 rows and 4 columns.
- `signup_date` is stored as text (object) needs conversion to datetime.
- there are no null values in any columns
- there is no duplicate values in `customer_id`


# Data Cleaning
Now, I will clean and prepare the data `dim_customer` for analysis:
1) convert `signup_date` to datetime datatype

In [113]:
# Make a working copy
customers_clean = customers.copy()


In [114]:
# Convert signup_date to datetime
customers_clean["signup_date"] = pd.to_datetime(customers_clean["signup_date"], errors="coerce", dayfirst=True)

In [115]:
#Standardise text columns
customers_clean["city"] = customers_clean["city"].str.strip().str.title()
customers_clean["acquisition_channel"] = customers_clean["acquisition_channel"].str.strip().str.title()


In [116]:
# Confirm data types
customers_clean.dtypes

customer_id                    object
signup_date            datetime64[ns]
city                           object
acquisition_channel            object
dtype: object

In [117]:
# Confirm text standardisation
customers_clean.head()

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,2025-03-21,Pune,Organic
1,CUST000008,2025-02-07,Kolkata,Referral
2,CUST000009,2025-02-25,Delhi,Paid
3,CUST000010,2025-01-28,Hyderabad,Paid
4,CUST000011,2025-01-18,Kolkata,Organic


### Observations
- `signup_date` successfully converted to datetime.
- Both `city` and `acquisition_channel` standardized.

# Save the Clean Dataset

In [118]:
# Save cleaned file in CSV format
customers_clean.to_csv(Clean_Data / "dim_customer_clean.csv", index=False)

# Some basic analysis

In [119]:
# Number of customers by city
customers_clean["city"].value_counts().head(10)


city
Bengaluru    30281
Mumbai       17317
Delhi        15090
Chennai      10823
Hyderabad    10755
Pune          8532
Ahmedabad     7512
Kolkata       7466
Name: count, dtype: int64

In [120]:
# Customers by acquisition channel
customers_clean["acquisition_channel"].value_counts()

acquisition_channel
Organic     55382
Paid        25730
Referral    15377
Social      11287
Name: count, dtype: int64

# Observations
- **Bengaluru** has most customers and most customers came from **Organic** `acquisition_channel`

# 3) Understanding and Cleaning the `dim_restaurant` Dataset

This dataset contains restaurant-level details such as name, city, cuisine type, partnership model,  
average preparation time, and active status.



In [121]:
# Load the restaurant dataset
restaurants = pd.read_csv(Raw_Data / "dim_restaurant.csv")

In [122]:
#display number of rows and columns  and top few rows
print("Shape:", restaurants.shape)
restaurants.head()


Shape: (19995, 7)


,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
0,REST12962,Flavours of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,N
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,Y
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,Y
3,REST19745,Delhi Express Palace,Ahmedabad,Pizza,Cloud Kitchen,16-25,Y
4,REST02383,Namma Mess Delight,Mumbai,Healthy,Restaurant,26-40,Y


In [123]:
# Check column datatypes 
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   restaurant_id      19995 non-null  object
 1   restaurant_name    19995 non-null  object
 2   city               19995 non-null  object
 3   cuisine_type       19995 non-null  object
 4   partner_type       19995 non-null  object
 5   avg_prep_time_min  19995 non-null  object
 6   is_active          19995 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [124]:
#display statistical data of each numeric columns
restaurants.describe()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
count,19995,19995,19995,19995,19995,19995,19995
unique,19995,4779,8,8,2,4,2
top,REST12962,Classic Tandoor Heaven,Bengaluru,North Indian,Restaurant,16-25,Y
freq,1,12,4963,3942,14028,7987,18050


In [125]:
#Check null values
restaurants.isna().sum()

restaurant_id        0
restaurant_name      0
city                 0
cuisine_type         0
partner_type         0
avg_prep_time_min    0
is_active            0
dtype: int64

In [126]:
restaurants["avg_prep_time_min"].unique()

array(['26-40', '16-25', '>40', '<=15'], dtype=object)

In [127]:
#Check for duplicates in restaurant_id"
duplicates_restaurant_id = restaurants["restaurant_id"].duplicated().sum()
print("Duplicate restaurant IDs:", duplicates_restaurant_id)

Duplicate restaurant IDs: 0


# Observations
- The `dim_restaurant` has 19995 rows and 7 columns.
- `is_active` contains yes/no values . we will convert it into boolean for easy analysis
- average_prep_time is stored as text (object) and nd has 4 bins <=15, 16-25, 25-40, >40. We will keep it as it is and will make another column average_prep_time_est where we wiconvert <=15 → 15  ,- 16–25 → 20.5 , - 26–40 → 33,  - >40 → .45 ll 
- there are no null values in any columns
- there is no duplicate values in restaurant_id

# Data Cleaning

In [128]:
# Make a working copy
restaurants_clean = restaurants.copy()

In [129]:
#  Standardize text columns (remove spaces, fix capitalization)
for col in ["restaurant_name", "city", "cuisine_type", "partner_type"]:
    restaurants_clean[col] = restaurants_clean[col].astype(str).str.strip().str.title()

In [130]:
#  Convert is_active to Boolean
restaurants_clean["is_active"] = (
    restaurants_clean["is_active"].astype(str).str.strip().str.upper().map({"Y": True, "N": False})
)

In [131]:
# Cleaning average prep time


# Convert to string and strip spaces
col = restaurants_clean["avg_prep_time_min"].astype(str).str.strip()

#  Handle ranges like "16-25" or "26–40"
ranges = col.str.extract(r'(\d+)\s*[-–]\s*(\d+)').astype(float)
range_mid = ranges.mean(axis=1)  # midpoint = (low + high)/2

#  Handle <=15
le = col.str.extract(r'<=\s*(\d+)')[0].astype(float) # choose this as we are keeping it as upper bound

# Handle >40
gt = col.str.extract(r'>\s*(\d+)')[0].astype(float)
# Optionally, assume next bin of 10 mins → midpoint +5
gt_mid = gt + 5

# Combine results
restaurants_clean["avg_prep_time_est"] = range_mid.fillna(le).fillna(gt_mid)


# Verify dataset after Cleaning

In [132]:
restaurants_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      19995 non-null  object 
 1   restaurant_name    19995 non-null  object 
 2   city               19995 non-null  object 
 3   cuisine_type       19995 non-null  object 
 4   partner_type       19995 non-null  object 
 5   avg_prep_time_min  19995 non-null  object 
 6   is_active          19995 non-null  bool   
 7   avg_prep_time_est  19995 non-null  float64
dtypes: bool(1), float64(1), object(6)
memory usage: 1.1+ MB


In [133]:
restaurants_clean.head()


,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active,avg_prep_time_est
0,REST12962,Flavours Of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,False,33.0
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,True,20.5
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,True,20.5
3,REST19745,Delhi Express Palace,Ahmedabad,Pizza,Cloud Kitchen,16-25,True,20.5
4,REST02383,Namma Mess Delight,Mumbai,Healthy,Restaurant,26-40,True,33.0


In [134]:

restaurants_clean.describe(include="all")

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active,avg_prep_time_est
count,19995,19995,19995,19995,19995,19995,19995,19995.000000
unique,19995,4779,8,8,2,4,2,NaN
top,REST12962,Classic Tandoor Heaven,Bengaluru,North Indian,Restaurant,16-25,True,NaN
freq,1,12,4963,3942,14028,7987,18050,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.687447
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.422877
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.500000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.000000


In [135]:
restaurants_clean.isna().sum()

restaurant_id        0
restaurant_name      0
city                 0
cuisine_type         0
partner_type         0
avg_prep_time_min    0
is_active            0
avg_prep_time_est    0
dtype: int64

In [136]:
# Quick check
restaurants_clean["avg_prep_time_min"].unique()

array(['26-40', '16-25', '>40', '<=15'], dtype=object)

In [137]:
restaurants_clean["avg_prep_time_est"].unique()

array([33. , 20.5, 45. , 15. ])

### Cleaning Notes- Observations
- Columns "restaurant_name", "city", "cuisine_type", "partner_type" are standardised
- Converted is_active to Boolean
- **avg_prep_time_min** is kept as it is but we made new column(`avg_prep_time_est`) for numeric values
  - `16–25` → midpoint = 20.5 
  - `26–40`→ midpoint = 33
  - `<=15` → 15  
  - `>40` → 45 

# Save the Clean Dataset

In [138]:
# Save cleaned file in CSV format
restaurants_clean.to_csv(Clean_Data / "dim_restaurant_clean.csv", index=False)

# 4) Understanding the `fact_delivery_performance` Dataset

This table contains delivery-level metrics (actual vs expected delivery times, and distance).  
We’ll first **load and inspect** the data to understand its shape, schema, and basic quality.
Then we will check row count, datatypes, missing values, and rough distributions.

In [4]:
#  Load the delivery performance dataset
delivery_performance = pd.read_csv(Raw_Data/ "fact_delivery_performance.csv")


In [5]:
#display number of rows and columns and first few rows
print("Shape:", delivery_performance.shape)
display(delivery_performance.head())

Shape: (149166, 4)


,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km
0,ORD202501023439,31,31,6.4
1,ORD202501012051,46,42,1.9
2,ORD202501019281,25,31,6.2
3,ORD202501000124,29,41,6.2
4,ORD202501006518,37,41,2.4


In [6]:
# Check column data types
delivery_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149166 entries, 0 to 149165
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   order_id                     149166 non-null  object 
 1   actual_delivery_time_mins    149166 non-null  int64  
 2   expected_delivery_time_mins  149166 non-null  int64  
 3   distance_km                  149166 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 4.6+ MB


In [7]:
# Missing values by column
delivery_performance.isna().sum()

order_id                       0
actual_delivery_time_mins      0
expected_delivery_time_mins    0
distance_km                    0
dtype: int64

In [8]:
#display statistical data of each numeric columns
delivery_performance.describe()

,actual_delivery_time_mins,expected_delivery_time_mins,distance_km
count,149166.000000,149166.000000,149166.000000
mean,44.404549,38.691679,4.494667
std,12.784224,5.074100,2.021213
min,25.000000,30.000000,1.000000
25%,36.000000,35.000000,2.700000
50%,42.000000,39.000000,4.500000
75%,50.000000,43.000000,6.200000
max,90.000000,50.000000,8.000000


# Observations
- The `dim_restaurant` has 149166 rows and 4 columns.
- there are no null values in any columns
- Since all time and distance fields are already numeric, 
no additional data type cleaningias required.

### Feature Engineering — Delivery Timeliness

To evaluate delivery performance, we created two key columns:

- **delay_mins** — Calculates the delay in minutes as  
  `actual_delivery_time_mins - expected_delivery_time_mins`.  
  - Positive → Delivered late  
  - Zero → On time  
  - Negative → Delivered early  

- **delivery_is_on_time** — Boolean flag that is `True` if the delivery met or beat the expected time (on-time), and `False` otherwise.

These metrics enable analysis of QuickBite’s delivery efficiency and SLA compliance, both overall and across partners, cities, and phases.


In [9]:
# Make a working copy
delivery_performance_clean = delivery_performance.copy()

In [10]:
# Create delay_mins column
delivery_performance_clean["delay_mins"] = ( delivery_performance_clean["actual_delivery_time_mins"] - delivery_performance_clean["expected_delivery_time_mins"])

In [11]:
# Create delivery_is_on_time column
delivery_performance_clean["Delivery_is_on_time"] = delivery_performance_clean["delay_mins"] <= 0

In [12]:
#check dataframe top few rows
delivery_performance_clean.head()

,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km,delay_mins,Delivery_is_on_time
0,ORD202501023439,31,31,6.4,0,True
1,ORD202501012051,46,42,1.9,4,False
2,ORD202501019281,25,31,6.2,-6,True
3,ORD202501000124,29,41,6.2,-12,True
4,ORD202501006518,37,41,2.4,-4,True


In [13]:
# Check column data types
delivery_performance_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149166 entries, 0 to 149165
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   order_id                     149166 non-null  object 
 1   actual_delivery_time_mins    149166 non-null  int64  
 2   expected_delivery_time_mins  149166 non-null  int64  
 3   distance_km                  149166 non-null  float64
 4   delay_mins                   149166 non-null  int64  
 5   Delivery_is_on_time          149166 non-null  bool   
dtypes: bool(1), float64(1), int64(3), object(1)
memory usage: 5.8+ MB


In [14]:
delivery_performance_clean.describe(include='all')

,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km,delay_mins,Delivery_is_on_time
count,149166,149166.000000,149166.000000,149166.000000,149166.000000,149166
unique,149166,NaN,NaN,NaN,NaN,2
top,ORD202501023439,NaN,NaN,NaN,NaN,False
freq,1,NaN,NaN,NaN,NaN,95226
mean,NaN,44.404549,38.691679,4.494667,5.712870,NaN
std,NaN,12.784224,5.074100,2.021213,12.327935,NaN
min,NaN,25.000000,30.000000,1.000000,-20.000000,NaN
25%,NaN,36.000000,35.000000,2.700000,-3.000000,NaN
50%,NaN,42.000000,39.000000,4.500000,4.000000,NaN
75%,NaN,50.000000,43.000000,6.200000,12.000000,NaN


### Observations — Delivery Data

- Average **actual delivery time** is around **44 minutes**, compared to an **expected time** of about **39 minutes**, indicating a minor average delay (~5 minutes).  
- Delivery distances range from **1 km to 8 km**, which is realistic for urban delivery operations.  
- The derived columns `delay_mins` and `delivery_is_on_time` were successfully created.  
- On average, deliveries are slightly delayed but mostly on time.  
The data appears clean, with no abnormal or extreme delivery times — ready for export to Power BI for further analysis.


In [15]:
# we will add delay bucket
def bucket_delay(x):
    if x <= 0:
        return "On Time / Early"
    elif x <= 10:
        return "0–10 min Delay"
    elif x <= 20:
        return "10–20 min Delay"
    else:
        return ">20 min Delay"

delivery_performance_clean["Delay_Bucket"] = delivery_performance_clean["delay_mins"].apply(bucket_delay)


# Save the clean dataset

In [16]:
# Save cleaned file in CSV format
delivery_performance_clean.to_csv(Clean_Data / "fact_delivery_performance_clean.csv", index=False)

# 5) Understanding the `fact_order_items` Dataset

This table contains **one row per item** inside an order. It links orders to menu items and captures quantity, pricing, and discounts.
We will load, confirm schema, check missing values, and validate price math.


In [151]:
# Load the order items dataset
order_items = pd.read_csv(Raw_Data / "fact_order_items.csv")


In [152]:
# Check shape and first few rows
print("Shape:", order_items.shape)
display(order_items.head())

Shape: (342994, 8)


,order_id,item_id,menu_item_id,restaurant_id,quantity,unit_price,item_discount,line_total
0,ORD202501006518,ITEM001,MENU12962_3216,REST12962,2,48.31,0.00,96.62
1,ORD202501006518,ITEM002,MENU12962_1962,REST12962,3,61.24,0.00,183.71
2,ORD202501019281,ITEM001,MENU14069_0510,REST14069,2,87.19,0.00,174.38
3,ORD202501019281,ITEM002,MENU14069_4308,REST14069,3,84.67,0.00,254.00
4,ORD202501023439,ITEM001,MENU08622_7574,REST08622,2,235.81,35.44,436.18


In [153]:
# check datatypes & non-null counts
order_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342994 entries, 0 to 342993
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       342994 non-null  object 
 1   item_id        342994 non-null  object 
 2   menu_item_id   342994 non-null  object 
 3   restaurant_id  342994 non-null  object 
 4   quantity       342994 non-null  int64  
 5   unit_price     342994 non-null  float64
 6   item_discount  342994 non-null  float64
 7   line_total     342994 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 20.9+ MB


In [154]:
# Missing values by column
order_items.isna().sum()

order_id         0
item_id          0
menu_item_id     0
restaurant_id    0
quantity         0
unit_price       0
item_discount    0
line_total       0
dtype: int64

In [155]:
#display statistical data of each numeric columns
order_items.describe()

,quantity,unit_price,item_discount,line_total
count,342994.000000,342994.000000,342994.000000,342994.000000
mean,1.998012,86.307888,9.193635,143.814273
std,0.818149,79.286627,17.089856,108.096155
min,1.000000,4.330000,0.000000,5.410000
25%,1.000000,39.350000,0.000000,66.340000
50%,2.000000,62.270000,0.000000,112.785000
75%,3.000000,102.230000,12.580000,190.150000
max,3.000000,900.000000,220.660000,900.000000


### Observations — `fact_order_items`

- No missing values detected.
- Data types correctly assigned (`int64` and `float64`).
- No negative or zero quantities found.
- Dataset is clean and ready for use in Power BI.

# Data Validation
The dataset fact_order_items has a column called line_total, which should theoretically represent:
line_total = (quantity * unit_price) - item_discount

Now, since all three components (quantity, unit_price, and item_discount) exist in the data, we can cross-verify whether the provided line_total values were calculated correctly — or if there are data entry or rounding errors.


In [156]:
#Validate price consistency

# Recalculate line total
order_items["calc_line_total"] = order_items["quantity"] * order_items["unit_price"] - order_items["item_discount"]

# Find mismatch rows (allowing a tiny rounding tolerance)
order_items["diff"] = (order_items["line_total"] - order_items["calc_line_total"]).round(2)
mismatch_count = (order_items["diff"].abs() > 0.01).sum()

print(f"Total mismatched rows (> 0.01 difference): {mismatch_count}")

# Display a few mismatched rows if any
if mismatch_count > 0:
    display(order_items.loc[order_items['diff'].abs() > 0.01].head())
else:
    print("✅ All line totals match the calculation!")


Total mismatched rows (> 0.01 difference): 0
✅ All line totals match the calculation!


# Validation Check: Observations
> Verified that `line_total = quantity × unit_price − item_discount`.  
> No discrepancies found, confirming pricing integrity.  

# Saving the Files

In [157]:
# Create clean version for Power BI
order_items_clean = order_items.copy()

# Drop helper columns
order_items_clean.drop(columns=["calc_line_total", "diff"], inplace=True, errors="ignore")

# Save clean file
order_items_clean.to_csv(Clean_Data / "fact_order_items_clean.csv", index=False)


### Saved the cleaned file in csv format and dropped the helper columns "calc_line_total", "diff" which we made for data validation

# 6) Understanding the `fact_ratings` Dataset

This table captures customer feedback at the **order** level. It will be central to measuring
trust and recovery post-crisis.

In [158]:
# Load and inspect ratings dataset
ratings = pd.read_csv(Raw_Data / "fact_ratings.csv")


In [159]:
# Display shape
print("Shape:", ratings.shape)


Shape: (68842, 7)


In [160]:
# Display first few rows
display(ratings.head())

,order_id,customer_id,restaurant_id,rating,review_text,review_timestamp,sentiment_score
0,ORD202501023439,CUST181110,REST08622,4.5,Super fast delivery,01-01-2025 15:00,0.75
1,ORD202501019281,CUST179306,REST14069,4.5,Great taste!,01-01-2025 15:00,0.75
2,ORD202501018036,CUST093042,REST13907,5.0,Super fast delivery,01-01-2025 14:03,1.00
3,ORD202501007724,CUST110825,REST08451,4.0,Tasty but a bit late,01-01-2025 14:06,0.50
4,ORD202501002349,CUST181669,REST00225,4.4,Satisfied overall,01-01-2025 14:06,0.70


In [161]:
#Check datatype and non null values
ratings.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68842 entries, 0 to 68841
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          68825 non-null  object 
 1   customer_id       68825 non-null  object 
 2   restaurant_id     68825 non-null  object 
 3   rating            68825 non-null  float64
 4   review_text       68825 non-null  object 
 5   review_timestamp  68825 non-null  object 
 6   sentiment_score   68825 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.7+ MB


In [162]:
# Check for missing values
ratings.isna().sum()


order_id            17
customer_id         17
restaurant_id       17
rating              17
review_text         17
review_timestamp    17
sentiment_score     17
dtype: int64

In [163]:
# check statistical values
ratings.describe()

,rating,sentiment_score
count,68825.000000,68825.000000
mean,4.052308,0.526154
std,0.940331,0.470165
min,1.000000,-1.000000
25%,3.800000,0.400000
50%,4.400000,0.700000
75%,4.700000,0.850000
max,5.000000,1.000000


# Observations
- Missing values: Only 17 rows have nulls (in all columns), so likely incomplete records we will analysis them in depth further	
- rating and sentiment_score are already numeric	
- review_timestamp is still object type → we will convert it to datetime	
- rating.describe(): mean ≈ 4.05 → indicates generally positive reviews	
- sentiment_score.describe(): mean ≈ 0.25 → aligns with positive text sentiment overall

# Data Cleaning 
- First we will do missing value analysis
- we will change the datatye of column review timestamp from text ( object) to datetime
- we will check if there is any invalid ratings ( not between 1-5)
- we will clean `review_text` column. we will remove trailing spaces, emojis, and special characters from review_text for consistency. Clean text helps for later Power BI sentiment or word frequency visuals.
- Validate sentiment vs rating ,ensuring the sentiment_score roughly aligns with rating (higher ratings → higher sentiment scores).

In [164]:
#Investigate missing rows

# Filter rows where any key column is null
missing_rows_ratings = ratings[
    ratings[["order_id", "customer_id", "restaurant_id", "rating", "review_text"]].isna().any(axis=1)
]

print(f"Total missing rows: {len(missing_rows_ratings)}")
display(missing_rows_ratings.head(10))


Total missing rows: 17


,order_id,customer_id,restaurant_id,rating,review_text,review_timestamp,sentiment_score
68812,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68815,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68816,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68826,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68828,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68830,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68831,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68832,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68833,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68834,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
# Check if they belong to cancelled or incomplete orders
merged = missing_rows_ratings.merge(orders_clean[["order_id", "is_cancelled"]], on="order_id", how="left")
merged["is_cancelled"].value_counts(dropna=False)


is_cancelled
NaN    17
Name: count, dtype: int64


### Upon investigation, all 17 missing rows contained **no data across any column** — 
these appear to be system-generated placeholders or incomplete uploads.

The missing ratings were merged with the `orders_clean` dataset to verify if they belonged to cancelled or incomplete orders

- None of the 17 rows had a matching `order_id` in the orders table.
- Therefore, they do not correspond to any valid transaction and are likely system artifacts
.

### Decision: Safely remove these rows from the dataset 

In [166]:
# creating a copy of ratings dataset
ratings_clean = ratings_clean.copy()

NameError: name 'ratings_clean' is not defined

In [ ]:
# Drop rows where all main columns are missing
ratings_clean = ratings.dropna(
    subset=["order_id", "customer_id", "restaurant_id", "rating"], how="any"
).copy()


In [ ]:
#verify if these is still null values
ratings_clean.isna().sum()

In [ ]:
#Convert review_timestamp to datetime
ratings_clean["review_timestamp"] = pd.to_datetime(
    ratings_clean["review_timestamp"], errors="coerce"
)

In [ ]:
#verify datatypes
ratings_clean.dtypes

In [ ]:
# Checking for invalid ratings
print("Unique ratings:", sorted(ratings_clean["rating"].unique()))


### There are no invalid ratings and all are between 1-5

In [ ]:
# Clean review_text column
ratings_clean["review_text"] = (
    ratings_clean["review_text"]
    .astype(str)
    .str.strip()
    .str.replace(r"[^a-zA-Z0-9\s.,!?]", "", regex=True)
)

In [ ]:
# Validate sentiment vs rating
ratings_clean[["rating", "sentiment_score"]].corr()

### The correlation between `rating` and `sentiment_score` is **1.0**, indicating a perfect positive relationship.

 # Save Cleaned Dataset

In [ ]:
# Save as CSV file format
ratings_clean.to_csv(Clean_Data / "fact_ratings_clean.csv", index=False)

# 7) Understanding `dim_delivery_partner` Dataset

The **Delivery Partner Dimension Table** contains details about delivery riders or partners associated with QuickBite.  
It provides information such as partner identity, city assignment, joining date, and active status. It will later help us in analyzing delivery performance, partner utilization, and operational coverage across cities.


In [ ]:
# Load delivery partner data
delivery_partners = pd.read_csv(Raw_Data/ "dim_delivery_partner_.csv")


In [ ]:
# Display first few rows
display(delivery_partners.head())


In [ ]:
# Shape and structure
print("Rows, Columns:", delivery_partners.shape)
delivery_partners.info()

In [ ]:
# Check missing values
delivery_partners.isna().sum()

In [ ]:
#Checking statistical information
delivery_partners.describe()

##  Initial Observations — `dim_delivery_partner`
- Total Rows: 15,000 Total Columns: 7
- No missing values were detected across any columns.
- The `is_active` column uses **Y/N flags**, which can be converted to Boolean (`True/False`) for easier analysis.  
- Text columns such as `city`, `vehicle_type`, and `employment_type` can be standardized (title case and trimmed spaces) to ensure uniformity.
- we need to add a dummy row for cancelled before assignment records which we made while doing missing value analysis of facts_orders table  


# Clean and Standardize dim_delivery_partner

### Handling “Cancelled Before Assignment” Records

In the `fact_orders` dataset, some orders were cancelled before a delivery partner could be assigned.  
These records have `delivery_partner_id = "Cancelled before assignment"`.

To maintain **referential integrity** between `fact_orders` and `dim_delivery_partner`,  
a **dummy record** is added to `dim_delivery_partner`.  

This ensures Power BI (or any BI tool) can create valid relationships **without excluding cancelled orders**.  
Since this entry does not represent a real delivery partner, most fields are left as `None`,  
and `is_active` is set to `False`.


In [ ]:
# Make a working copy
delivery_partners_clean = delivery_partners.copy()

In [ ]:
# Add dummy record for 'Cancelled before assignment' orders

# Check if such orders exist in fact_orders_clean
if "Cancelled before assignment" in orders_clean["delivery_partner_id"].values:
    
    # Define dummy record
    dummy_row = {
        "delivery_partner_id": "Cancelled before assignment",
        "partner_name": "Not Assigned",
        "city": None,
        "employment_type": None,
        "is_active": False,
        "vehicle_type": None,
        "avg_rating": None
    }
    
    # Append only if not already present
    if "Cancelled before assignment" not in delivery_partners_clean["delivery_partner_id"].values:
        delivery_partners_clean = pd.concat(
            [delivery_partners_clean, pd.DataFrame([dummy_row])],
            ignore_index=True
        )
        print("✅ Dummy delivery partner added for 'Cancelled before assignment' orders.")
    else:
        print("ℹ️ Dummy record already exists in dim_delivery_partner.")
else:
    print("ℹ️ No 'Cancelled before assignment' orders found in fact_orders.")


In [ ]:
# Convert is_active column to Boolean for easier analysis
delivery_partners_clean["is_active"] = (
    delivery_partners_clean["is_active"].map({"Y": True, "N": False})
)

In [ ]:
# Standardize text columns
text_cols = ["partner_name", "city", "vehicle_type", "employment_type"]
for col in text_cols:
    delivery_partners_clean[col] = delivery_partners_clean[col].str.strip().str.title()

In [ ]:
# Quick data validation
print("Unique vehicle types:", delivery_partners_clean["vehicle_type"].unique())
print("Unique employment types:", delivery_partners_clean["employment_type"].unique())
print("Rating range:", delivery_partners_clean["avg_rating"].min(), "-", delivery_partners_clean["avg_rating"].max())

## Observations
- Added a dummy row for "cancelled before assignment" `delivery_partner_id`
- Converted `is_active` values** from `Y/N` to Boolean (`True/False`) for easier filtering and calculations.  
- Standardized text columns** (`partner_name`, `city`, `vehicle_type`, `employment_type`)  .  
- Confirmed `avg_rating` lies within expected range (1–5).  


# Saving Dataset

In [ ]:
# Save the cleaned datasetas csv format
delivery_partners_clean.to_csv(Clean_Data / "dim_delivery_partner_clean.csv", index=False)


## 8) Understanding `dim_menu_items` Dataset

The purpose of this dataset is to provide detailed information about all menu items offered by restaurants on the QuickBite platform.  
It serves as a key lookup table that helps connect customer orders to specific food items and their pricing details.

In [ ]:
# Load the dataset
menu_items = pd.read_csv(Raw_Data/ "dim_menu_item.csv")

In [ ]:
# Display first few rows
menu_items.head()



In [ ]:
# Check shape and data types
print("Shape:", menu_items.shape)
menu_items.info()

In [ ]:
# Check for missing values
menu_items.isna().sum()

In [ ]:
# Quick statistics for numeric columns
menu_items.describe()

# Initial Observations — dim_menu_items¶
- Total Rows: 342671 Total Columns: 6
- No missing values were detected across any columns.
- The is_veg column uses Y/N flags, which can be converted to Boolean (True/False) for easier analysis.
- Text columns category, item_name can be standardized (title case and trimmed spaces) to ensure uniformity.

# Clean and Standardize dim_menu_items

In [ ]:
# Make a working copy
menu_items_clean = menu_items.copy()

In [ ]:
#  Convert is_veg column to Boolean (True/False)
menu_items_clean["is_veg"] = (
    menu_items_clean["is_veg"].astype(str).str.strip().str.upper().map({"Y": True, "N": False})
)

In [ ]:
# Standardize text formatting for readability
for col in ["item_name", "category"]:
    menu_items_clean[col] = menu_items_clean[col].astype(str).str.strip().str.title()



In [ ]:
#checking for invalid price (negative or zero)
menu_items_clean[menu_items_clean["price"] <= 0]


In [ ]:
# Validate results
menu_items_clean.info()
menu_items_clean.head()



## Observations
- Converted `is_veg` values** from `Y/N` to Boolean (`True/False`) for easier filtering and calculations.  
- Standardized text columns** ("item_name", "category")  .  
- Confirmed `price` column has no invalid data.  


# Saving Dataset

In [ ]:
#  Save the cleaned dataset as csv file
menu_items_clean.to_csv(Clean_Data / "dim_menu_item_clean.csv", index=False)

## Creating `dim_date` — The Date Dimension

### Objective
Although no raw dataset for dates was provided, a **Date Dimension** (`dim_date`) is an essential because acts as a **calendar lookup table** that allows consistent time-based analysis across all fact tables (orders, deliveries, ratings, etc.).

Without it, Power BI and other BI tools cannot easily group data by **month**, **quarter**, or **year**, or calculate period-based trends.

### The date dimension enables:
-  **Time-series analysis** — e.g., daily, weekly, monthly trends.  
-  **Comparative analysis** — e.g., pre-crisis vs crisis vs recovery phases.  
-  **Dynamic filtering** — easy slicers in Power BI for year, month, quarter, or day.  
-  **Consistent joins** 


In [ ]:
# — Create Date Dimension

# Define the date range 
start_date = min(
    orders_clean["order_timestamp"].min(),
    customers_clean["signup_date"].min(),
    ratings_clean["review_timestamp"].min()
).date()

end_date = max(
    orders_clean["order_timestamp"].max(),
    customers_clean["signup_date"].max(),
    ratings_clean["review_timestamp"].max()
).date()

date_range = pd.date_range(start=start_date, end=end_date)
print(f"📅 Date Dimension Range: {start_date} → {end_date}")


In [ ]:
# Build the Date Dimension dataframe
dim_date = pd.DataFrame({
    "date": date_range,
    "year": date_range.year,
    "month": date_range.month,
    "month_name": date_range.strftime("%B"),
    "week": date_range.isocalendar().week,
    "weekday": date_range.weekday + 1,  # Monday=1, Sunday=7
    "weekday_name": date_range.strftime("%A"),
    "quarter": date_range.quarter,
    "is_weekend": date_range.weekday >= 5  # Sat/Sun = True
})

In [ ]:
dim_date.head()

In [ ]:
# Save the dataset
dim_date.to_csv(Clean_Data / "dim_date.csv", index=False)


In [ ]:
## 📆 Validating Date Coverage for `dim_date`

### 🎯 Objective
Before finalizing the `dim_date` (date dimension) table, we need to ensure that it covers the **entire timeline** present in all datasets.  
If the earliest or latest date in our fact tables falls outside the current range (`2025-01-01` → `2025-12-31`),  
it may cause **missing joins** or **incomplete visuals** in Power BI.

---

### ⚙️ Approach
We check the **minimum and maximum timestamps** across key tables:
- `fact_orders` → based on `order_timestamp`
- `fact_ratings` → based on `review_timestamp`
- `dim_customer` → based on `signup_date`

This helps verify:
1. Whether the existing date range (Jan–Dec 2025) is sufficient, or  
2. If the `dim_date` table should be **expanded** (e.g., to include late 2024 or early 2026).

---

### 🧠 Why It Matters
A properly ranged `dim_date` ensures:
- Accurate time-based relationships across all tables.
- Complete trend analysis in Power BI (no missing days/months).
- Reliable filters and slicers for dashboard users.

---

### ✅ Expected Outcome
- Print statements showing **min and max dates** per dataset.
- Confirmation that all dates fall within the `dim_date` range.
- If needed, dynamically adjust the start/end date to ensure full coverage.


## 🧾 Summary — `dim_date` Creation

### 🎯 Objective
To build a unified date reference table that supports time-based analysis and relationships between multiple fact tables (orders, ratings, deliveries, etc.) in Power BI.

### ⚙️ Steps Performed
1. **Generated a continuous range of dates** using `pandas.date_range()` for the entire year 2025.  
   (The range can be expanded to cover more years if needed.)  
2. **Derived key calendar attributes** such as:
   - `year`, `month`, `month_name`, `quarter`
   - `week`, `weekday`, `weekday_name`, `is_weekend`
3. **Created a structured DataFrame (`dim_date`)** to serve as a standard calendar dimension.  
4. **Saved the dataset** as `dim_date.csv` in the `data/clean` directory.

### ✅ Outcome
- A well-structured **Date Dimension** ready to be used as a master calendar table in Power BI.  
- Enables:
  - Monthly, weekly, and quarterly trend analysis.  
  - Time-based comparisons (Pre-crisis, Crisis, Recovery).  
  - Easy date filtering and slicing in dashboards.  

📁 **File saved:** `data/clean/dim_date.csv`


# 🧾 Final Summary — Data Cleaning & Preparation

### 🎯 Objective
To clean, validate, and prepare all datasets required for analyzing QuickBite’s crisis recovery performance.  
The cleaned data will serve as the foundation for creating an analytical dashboard in Power BI.

---

## 📊 Summary of Processed Datasets

| Dataset | Type | Key Cleaning Tasks | Output File |
|----------|------|--------------------|--------------|
| **fact_orders** | Fact | Converted timestamps, mapped Y/N to Boolean, added `phase` column (Pre-crisis / Crisis / Recovery) | `fact_orders_clean.csv` |
| **fact_order_items** | Fact | Verified price consistency (`line_total = qty × price − discount`) | `fact_order_items_clean.csv` |
| **fact_delivery_performance** | Fact | Added `delay_mins`, derived `delivery_is_on_time` | `fact_delivery_performance_clean.csv` |
| **fact_ratings** | Fact | Removed blank rows, converted dates, cleaned text, validated correlation between `rating` & `sentiment_score` | `fact_ratings_clean.csv` |
| **dim_customer** | Dimension | Cleaned city & acquisition channel text, ensured proper datetime format | `dim_customer_clean.csv` |
| **dim_restaurant** | Dimension | Parsed `avg_prep_time`, standardized names, handled nulls | `dim_restaurant_clean.csv` |
| **dim_delivery_partner** | Dimension | Converted Y/N → Boolean, standardized text fields, validated rating range | `dim_delivery_partner_clean.csv` |
| **dim_menu_item** | Dimension | Converted `is_veg` → Boolean, cleaned text, validated prices | `dim_menu_item_clean.csv` |
| **dim_date** | Dimension | Created from scratch to enable time-based analysis | `dim_date.csv` |

---

### 🧠 Key Takeaways
- All datasets are **clean, validated, and properly typed**.  
- File naming and folder structure follow a professional pattern (`data/raw` → `data/clean`).  
- Consistent variable naming and data types ensure smooth integration in Power BI.  
- A complete **star schema model** can now be built in Power BI using:
  - **Fact Tables:** Orders, Items, Ratings, Delivery Performance  
  - **Dimension Tables:** Customers, Restaurants, Delivery Partners, Menu Items, Date  

---

### ✅ Next Steps
1. **Build the Power BI data model** (establish relationships between fact and dimension tables).  
2. **Create calculated columns/measures** for KPIs:
   - Order trends, Delivery delays, Customer churn, Campaign recovery impact, etc.  
3. **Design the recovery dashboard** — combining visual storytelling with business insights.  
4. **Prepare a short video presentation** explaining your findings and recommendations.

---

### 💬 Reflection
This notebook demonstrates:
- End-to-end data cleaning & preparation.  
- Understanding of data modeling best practices (star schema).  
- Readiness to perform EDA, visualization, and storytelling in Power BI.  

📁 All cleaned datasets are stored in:  
**`data/clean/`**



In [ ]:
🗂️ **Note on Power BI Table Naming**
All cleaned CSVs were imported into Power BI from `data/clean/`.  
For clarity, the `_clean` suffix was removed after import:
- `fact_orders_clean` → `fact_orders`
- `dim_customer_clean` → `dim_customer`
- `fact_delivery_performance_clean` → `fact_delivery_performance`

These renames do not affect the original CSV files.
